In [1]:
import gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation as LDA
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
target_category = [
    38,  # アイスクリーム__ノベルティー
    110,  # スナック・キャンディー__ガム
    113,  # スナック・キャンディー__シリアル
    114,  # スナック・キャンディー__スナック
    134,  # チョコ・ビスクラ__チョコレート
    171,  # ビール系__RTD
    172,  # ビール系__ノンアルコール
    173,  # ビール系__ビール系
    376,  # 和菓子__米菓
    435,  # 大型PET__無糖茶（大型PET）
    467,  # 小型PET__コーヒー（小型PET）
    537,  # 水・炭酸水__大型PET（炭酸水）
    539,  # 水・炭酸水__小型PET（炭酸水）
    629,  # 缶飲料__コーヒー（缶）
    768,  # 麺類__カップ麺
]

In [5]:
# 2020-01-01以降、2020-08-01以前で10分以上経過し購買が発生したセッションにtrainデータを絞る
tmp_sessions = meta[(meta['date'] >= '2020-01-01')&(meta['date'] < '2020-08-01')]['session_id'].unique()
tmp_log = cartlog[cartlog["session_id"].isin(tmp_sessions)]
print('2020-01-01以降、2020-08-01以前: ', len(tmp_sessions))

# 購買が発生したセッション
payment_sessions = set(tmp_log[tmp_log['is_payment']==1]['session_id'].unique())
print('購買が発生: ', len(payment_sessions))
# 10分以上のセッション
over10min_sessions = set(tmp_log[tmp_log['spend_time']>=600]['session_id'].unique())
print('10分以上: ', len(over10min_sessions))
# 積集合
all_train_sessions = payment_sessions & over10min_sessions
print('積集合: ', len(all_train_sessions))

# 10分以上の全trainのログデータ
all_train_log = tmp_log[tmp_log["session_id"].isin(all_train_sessions)]
print('全trainのログ: ', len(all_train_log))

2020-01-01以降、2020-08-01以前:  234103
購買が発生:  229959
10分以上:  174412
積集合:  172868
全trainのログ:  4548534


In [6]:
def agg_payment(cartlog) -> pd.DataFrame:
    """セッションごと・商品ごとの購買個数を集計する"""
    # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
    agg = cartlog.loc[cartlog["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    return pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner").drop(columns=['JAN'])

In [7]:
class RetailDataset:
    def __init__(self, thres_sec, meta):
        self.thres_sec = thres_sec
        self.meta = meta.copy()
        self.meta['time_elapsed_sec'] = self.meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        
        # all_train_logの中で、指定時間より前のログのみを抽出　-> public_train_log, train_sessions
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        self.public_train_log = merge_train[merge_train['spend_time'] <= merge_train['time_elapsed_sec']]
        self.train_sessions = self.public_train_log["session_id"].unique()
        
    def get_train_input_log(self) -> pd.DataFrame:
        return self.public_train_log

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        train_target = pd.DataFrame(
            index=self.train_sessions,
        )
        train_target.index.name = "session_id"

        # time_elapsed以降のデータから購買個数を集計する
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        after_elapsed_log = merge_train[merge_train['spend_time'] > merge_train['time_elapsed_sec']]
        
        train_item_num = agg_payment(after_elapsed_log)
        train_item_num = train_item_num[train_item_num['category_id'].isin(target_category)]
        train_target_pos = train_item_num.groupby(["session_id", "category_id"])["n_items"].sum().unstack().fillna(0).astype(int)
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        return train_target.join(train_target_pos).fillna(0).reset_index()

In [8]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [9]:
ELAPSED_MIN = [0, 3, 5, 10]
# ELAPSED_MIN = [5]

In [10]:
LOG_VER = 4

In [11]:
def save_train_log(ver):
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        train_y.to_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))

In [12]:
def load_train_log(ver):
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        y = pd.read_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [13]:
# save_train_log(LOG_VER)

#### ver.4: 2020-01-01以降、2020-08-01以前
- 0, train_session (170752, 16)
- 3, train_session (172654, 16)
- 5, train_session (172794, 16)
- 10, train_session (172835, 16)

#### ver.3: 2020-04-01以降、2020-08-01以前
- 0, train_session (101562, 16)
- 3, train_session (102348, 16)
- 5, train_session (102394, 16)
- 10, train_session (102410, 16)

#### ver.2: 2020-08-01以前
- 0, train_session (378594, 16)
- 3, train_session (389649, 16)
- 5, train_session (390621, 16)
- 10, train_session (391074, 16)

### 過去のログデータ

In [14]:
payed_item = agg_payment(all_train_log)

In [15]:
def LDA_topic(df_input, topic, index, prefix):
    df_cp = df_input.set_index(index)
    lda = LDA(n_components=topic)
    lda_out = pd.DataFrame(lda.fit_transform(df_cp), index=df_cp.index).add_prefix(prefix)
    return lda_out.reset_index()

### ユーザ情報

In [16]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left")
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

In [17]:
def get_user_item(payed_item):
    # train用のデータから購買した商品
    user_payed_item = pd.merge(payed_item, meta[['session_id', 'user_id']], on='session_id', how='left')
    group_user_item = user_payed_item.groupby(['user_id', 'category_id'])[['n_items']].sum().reset_index()
    pivot_user_item = group_user_item.pivot_table(index='user_id', columns='category_id', values='n_items')
    # 全ユーザーの購入数の合計が5000以上のカテゴリに絞り込み
    sum_user_item = pivot_user_item.sum()
    user_item_index = sum_user_item[sum_user_item > 5000].index
    pivot_user_item = pivot_user_item[user_item_index].fillna(0).reset_index()
    # trainに存在しない人用に平均値で穴埋め
    user_item_mean = pivot_user_item.mean()
    # 全ユーザーとマージ
    all_user_item = pd.merge(user_master[['user_id']], pivot_user_item, on='user_id', how='left')
    # targetのカテゴリは除く
    for col in all_user_item.columns:
        if (col == 'user_id') or (col in target_category):
            continue
        new_col = 'user_pay_{}'.format(col)
        all_user_item[new_col] = all_user_item[col].fillna(user_item_mean[col]).astype('float32')
        all_user_item.loc[all_user_item[new_col]<0, new_col] = 0

    return all_user_item.drop(columns=list(user_item_mean.index))

In [18]:
all_user_item = get_user_item(payed_item)
print(all_user_item.shape)

(40350, 153)


In [19]:
def save_item_lda(ver, all_user_item):
    user_lda = LDA_topic(all_user_item, 10, 'user_id', 'LDA_')
    user_lda.to_feather('../inputs/user_lda_{}.f'.format(ver))

In [20]:
def load_item_lda(ver):
    user_lda = pd.read_feather('../inputs/user_lda_{}.f'.format(ver))
    return user_lda

In [21]:
# %%time
# save_item_lda(LOG_VER, all_user_item)

In [22]:
user_lda = load_item_lda(LOG_VER)
all_user_item = pd.merge(all_user_item, user_lda, on='user_id', how='left')
print(all_user_item.shape)

(40350, 163)


### メタ情報

In [23]:
def get_meta_features(meta):
    meta_features = meta.copy()
    meta_features['year'] = meta_features['date'].dt.year
    meta_features['month'] = meta_features['date'].dt.month
    meta_features['day'] = meta_features['date'].dt.day
    meta_features['dow'] = meta_features['date'].dt.dayofweek
    meta_features['doy'] = meta_features['date'].dt.dayofyear
    meta_features['week_time'] = meta_features['dow'] * 24 + meta_features['hour']
    le = preprocessing.LabelEncoder()
    meta_features['userid'] = le.fit_transform(meta_features['user_id'])
    
    # 曜日x時間の来店者数
    df_tz = meta_features.groupby(['week_time']).size().rename('timezone_count')
    df_tz =  pd.DataFrame(df_tz).reset_index()
    meta_features = pd.merge(meta_features, df_tz, on='week_time', how='left')
    
    # userごとに前の来店からどれくらいの日数が経ったか
    df = meta_features.groupby(['session_id', 'user_id', 'date']).first().reset_index().sort_values(['user_id', 'session_id'])
    df['date_diff'] = df['date'].diff(1)
    df['user_diff'] = df['user_id'].shift(1)
    df.loc[df['user_diff'] != df['user_id'], 'date_diff'] = pd.NaT
    df['date_diff'] = df['date_diff'].dt.days
    meta_features = meta_features.join(df[['date_diff']])

    # userごとの来店回数
    meta_features['date_rank'] = meta_features.groupby(['user_id'])['date'].rank(ascending=True)
    
    return meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'])

In [24]:
meta_features = get_meta_features(meta)

### ディスプレイアクション

In [25]:
disp_name_dic = {}
for i, disp in enumerate(display_action_id['display_name'].unique()):
    disp_name_dic[disp] = 'disp_cnt_{}'.format(i)
    
act_name_dic = {}
for i, action in enumerate(display_action_id['action_name'].unique()):
    act_name_dic[action] = 'act_cnt_{}'.format(i)

In [26]:
def get_display_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    disp_group_count = merge.groupby(['session_id', 'display_name']).size().reset_index().rename(columns={0:'disp_name_count'})
    disp_name_pivot = disp_group_count.pivot_table(index='session_id', columns='display_name', values='disp_name_count', aggfunc='sum')
    disp_name_pivot = disp_name_pivot.reset_index().fillna(0).rename(columns=disp_name_dic)
    
    disp_out = disp_name_pivot[['session_id']].copy()
    for val in disp_name_dic.values():
        disp_out[val] = 0
    
    for col in disp_name_pivot.columns:
        if col == 'session_id':
            continue
        disp_out[col] = disp_name_pivot[col]
    
    return disp_out

In [27]:
def get_action_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    act_group_count = merge.groupby(['session_id', 'action_name']).size().reset_index().rename(columns={0:'act_name_count'})
    act_name_pivot = act_group_count.pivot_table(index='session_id', columns='action_name', values='act_name_count', aggfunc='sum')
    act_name_pivot = act_name_pivot.reset_index().fillna(0).rename(columns=act_name_dic)
    
    act_out = act_name_pivot[['session_id']].copy()
    for val in act_name_dic.values():
        act_out[val] = 0
    
    for col in act_name_pivot.columns:
        if col == 'session_id':
            continue
        act_out[col] = act_name_pivot[col]
    
    return act_out

### セッション単位の特徴量

In [28]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'value_1']].rename(columns={'value_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, lda_coupon, on='coupon', how='left').drop(columns=['coupon'])
#     session_coupon = session_coupon.groupby(['session_id']).max().reset_index()
#     return session_coupon

In [29]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'name_1']].rename(columns = {'name_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, df_coupon_cat, on='coupon', how='left').drop(columns=['coupon'])
    
#     for cat in df_coupon_cat['coup_cat'].unique():
#         new_col = 'coup_cat_{}'.format(cat)
#         session_coupon[new_col] = 0
#         session_coupon.loc[session_coupon['coup_cat'] == cat, new_col] = 1
    
#     session_coupon.drop(columns=['coup_cat'], inplace=True)
#     return session_coupon.groupby('session_id').sum().reset_index()

In [30]:
def get_pre_payment_item(input_log):
    session_unique = input_log['session_id'].unique()
    agg = input_log.loc[input_log["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    agg = pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    agg = agg[agg['category_id'].isin(target_category)]
    agg = agg.groupby(["session_id", "category_id"])["n_items"].sum().reset_index()
    
    sesi = np.zeros(len(target_category))
    cate = [ct for ct in target_category]
    
    dummy = pd.DataFrame({'session_id':sesi, 'category_id':cate, 'n_items':sesi})
    agg = pd.concat([agg, dummy])
    
    agg = agg.pivot_table(index='session_id', columns='category_id', values='n_items').fillna(0)
    src_columns = ['x_{}'.format(c) for c in agg.columns]
    agg.columns = src_columns
    
    col = ['pre_target_{}'.format(c) for c in target_category]
    df_out = pd.DataFrame(index=session_unique, columns=col)
    df_out.index.name = "session_id"
    df_out = df_out.join(agg)
    for ct in target_category:
        src = 'x_{}'.format(ct)
        dst = 'pre_target_{}'.format(ct)
        df_out[dst] = df_out[src]
    
    return df_out.drop(columns=src_columns).fillna(0).reset_index()

In [31]:
def get_session_kind_group(input_log):
    kind_name ={
        'クーポン': 'coupon',
        '会計': 'kaikei',
        'キー': 'key',
        'カテゴリ': 'categry',
        'バーコードスキャン': 'barcode',
        'UUID': 'uuid',
        '使用ポイント': 'usedpoint',
        '確認': 'confirm',
        'ブランドスイッチ': 'bland',
        'レシピ': 'recipe',
        'スマホスキャン': 'smartphone',
        '磁気スキャン': 'magnetic',
        'レコメンド': 'recommend',
        '倍率ポイント': 'point',
    }
    group_count = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename('group_count_'+'item')
    for kind, name in kind_name.items():
        tmp = input_log[input_log['kind_1'] == kind].groupby(["session_id"]).size().rename('group_count_'+name)
        group_count = pd.concat([group_count, tmp], axis=1)
        
    return group_count.reset_index()

In [32]:
def get_session_item_info(input_log):
    item_log = input_log[input_log['kind_1'] == '商品'].copy()
    item_log = item_log.rename(columns={"value_1": "JAN"})
    item_log = pd.merge(item_log, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    item_log['total'] = item_log['n_items'] * item_log['unit_price']
    session_item = item_log.groupby(['session_id']).agg({
        'total':'sum', 
        'number_1':'sum', 
        'n_items':'sum', 
        'name_1':'nunique',
        'category_id':'nunique',
    }).reset_index()
    session_item = session_item.rename(columns={
        'number_1':'cart_item_cnt', 
        'n_items':'total_item_cnt', 
        'name_1':'item_nunique',
        'category_id':'cat_nunique',
    })
    session_item['mean_price'] = session_item['total'] / session_item['total_item_cnt']
    session_item['item_cnt_per_nuniq'] = session_item['total_item_cnt'] / session_item['item_nunique']
    session_item['item_nuniq_per_cat_nuniq'] = session_item['item_nunique'] / session_item['cat_nunique']
    session_item['mean_price_per_cat'] = session_item['total'] / session_item['cat_nunique']
    return session_item

In [33]:
def get_session_info(input_log):
    # アクション数
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    # 経過時間の平均
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    # ユニークユーザー
    unique_user = input_log.groupby(["session_id"])['user_id'].nunique().rename("uniq_user")
    
    session_features = pd.concat([
        n_actions,
        mean_spend_time,
        unique_user,
    ], axis=1)
    
    session_features['n_actions_user'] = session_features['n_actions'] * session_features['uniq_user']
    session_features['spend_time_user'] = session_features['spend_time'] * session_features['uniq_user']
    
    return session_features.reset_index()

### セッション単位で集計

In [34]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    
    session_feat = [
        get_session_info(input_log),
        get_session_kind_group(input_log),
        get_display_name_feature(input_log),
        get_action_name_feature(input_log),
        get_pre_payment_item(input_log),
#         get_coupon_info(input_log),
        get_session_item_info(input_log),
    ]
    
    for feat in session_feat:
        df_ses = pd.merge(df_ses, feat, on='session_id', how='outer')
        
    return df_ses

### 特徴量を集約する

In [35]:
def merge_features(input_log, session):
    feat_list = [
        get_session_features(input_log),
        user_features,
        meta_features,
    ]
    out = pd.DataFrame({"session_id": session})
    for feat in feat_list:
        out = pd.merge(out, feat, on="session_id", how="left")
        
    # userの情報
    out = pd.merge(out, all_user_item, on='user_id', how='left').drop(columns='user_id')

    assert len(session) == len(out)
    return out

In [36]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_features', train_features.shape)
    return train_features, y_train

In [37]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_features', test_features.shape)
    return test_features

In [38]:
n_fold = 4

In [39]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
#   'max_depth': 6,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

In [40]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [41]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [42]:
train_log_list, train_y_list = load_train_log(LOG_VER)

In [43]:
gc.collect()

38

In [44]:
%%time
df_pred_all = pd.DataFrame()
df_score_all = pd.DataFrame(index=ELAPSED_MIN)
models_list_list = []

for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
        df_score_all.loc[elapsed_min, target] = score
        
    models_list_list.append(models_list)
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_features (170752, 267)
test_features (14277, 267)
---- id = 38 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[544]	valid_0's auc: 0.813945
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[588]	valid_0's auc: 0.813741
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[493]	valid_0's auc: 0.813176
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[450]	valid_0's auc: 0.817966
--- FINISHED \ whole score: 0.8146 ---
---- id = 110 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[148]	valid_0's auc: 0.792113
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[169]	valid_0's auc: 0.786159
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[107]	valid_0's auc: 0.770315
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[112]	valid_0's auc: 0.794587
--- FINISHED \ whole score: 0.7850 ---
---- id = 113 -----


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[393]	valid_0's auc: 0.818003
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[173]	valid_0's auc: 0.785565
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[162]	valid_0's auc: 0.811403
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[310]	valid_0's auc: 0.806807
--- FINISHED \ whole score: 0.8039 ---
---- id = 629 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[117]	valid_0's auc: 0.839282
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[100]	valid_0's auc: 0.829526
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[230]	valid_0's auc: 0.833797
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[215]	valid_0's auc: 0.786526
--- FINISHED \ whole score: 0.8080 ---
---- id = 768 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[154]	valid_0's auc: 0.704033
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[193]	valid_0's auc: 0.712821
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[147]	valid_0's auc: 0.713957
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[213]	valid_0's auc: 0.703032
--- FINISHED \ whole score: 0.7084 ---
56486
CPU times: user 8h 58min 22s, sys: 4h 17min 19s, total: 13h 15min 41s
Wall time: 36min 38s


In [45]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.094116,0.022670,0.026952,0.276066,0.047307,0.008706,0.000894,0.026926,0.077051,0.007652,0.013808,0.003341,0.033089,0.010003,0.034725
663761,0.128337,0.037048,0.270877,0.392212,0.315296,0.127834,0.319519,0.043003,0.331967,0.004398,0.010168,0.007557,0.348603,0.002732,0.403489
663763,0.142940,0.060289,0.022776,0.901766,0.469189,0.007069,0.004298,0.002924,0.236244,0.016865,0.067777,0.000634,0.020193,0.026047,0.051685
663775,0.067963,0.007432,0.021626,0.240996,0.218549,0.054220,0.019481,0.176781,0.069363,0.189389,0.054192,0.004988,0.041149,0.005890,0.180141
663778,0.160731,0.014040,0.108985,0.186003,0.157897,0.074124,0.008139,0.050118,0.032763,0.039338,0.006160,0.013277,0.061985,0.009929,0.026532


In [46]:
df_score_all

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0.814566,0.785008,0.774916,0.755120,0.738231,0.856851,0.861703,0.854740,0.751050,0.870869,0.861627,0.894394,0.845662,0.898355,0.736641
3,0.807630,0.765901,0.769920,0.747263,0.730107,0.846242,0.845251,0.841813,0.745310,0.851663,0.848830,0.882076,0.830259,0.880864,0.722012
5,0.806708,0.757653,0.765539,0.742619,0.725872,0.839302,0.844170,0.834342,0.740124,0.838436,0.840640,0.866898,0.822385,0.852500,0.711663
10,0.803433,0.730796,0.758580,0.747569,0.736400,0.820980,0.815334,0.824849,0.752156,0.829601,0.825490,0.859537,0.803939,0.808038,0.708364


In [47]:
cv = df_score_all.mean(axis=1)
print(cv)
print('- cv =', cv.mean())

0     0.819982
3     0.807676
5     0.799257
10    0.788338
dtype: float64
- cv = 0.8038132158288026


In [48]:
assert len(df_pred_all) == len(test)

In [49]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [50]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)

#### baseline_14: 2020-01-01以降
- feat = 267
- Wall time: 36min 38s
- cv = 0.80381 (0.819/0.807/0.799/0.788)
- LB = 0.7473

#### baseline_13: 2020-04-01以降
- feat = 208
- Wall time: 25min 3s
- cv = 0.78581 (0.803/0.788/0.779/0.772)
- LB = 0.7304

#### baseline_12: 特徴量削減, lr=0.05
- feat = 345
- Wall time: 2h 18min 19s
- cv = 0.82506 (0.835/0.826/0.821/0.816)
- LB = 0.7675

#### baseline_11: クーポンカテゴリの種類減らす, ユニークユーザー数, 平均単価, ユニーク商品数 
- feat = 375
- Wall time: 2h 18min 19s
- cv = 0.81843 (0.831/0.820/0.814/0.806)
- LB = 0.7626

#### baseline_10: クーポンカテゴリの表示回数
- feat = 412
- Wall time: 2h 8min 53s
- cv = 0.821003 (0.831/0.823/0.818/0.8103)
- LB = 0.7657

#### baseline_9-1: 'feature_fraction': 0.6,'bagging_fraction': 0.6,'bagging_freq': 2,
- feat = 333
- Wall time: 2h 2min 14s
- cv = 0.81961 (0.829/0.822/0.816/0.809)
- LB = 0.7658

#### baseline_9: 来店間隔、回数、曜日x時間の来店者数
- feat = 333
- Wall time: 2h 9min 51s
- cv = 0.82118 (0.831/0.824/0.818/0.810)
- LB = 0.7665

#### baseline_8: クーポン発行とターゲットの購買実績
- feat = 344
- Wall time: 2h 5min 13s
- cv = 0.820398 (0.832/0.822/0.817/0.809)
- LB = 0.7484

#### baseline_7: userごとのカテゴリの過去の購買実績
- feat = 329
- Wall time: 1h 59min 16s
- cv = 0.82148 (0.832/0.824/0.818/0.810)
- LB = 0.7671

#### baseline_6: sesssion内のターゲットの購買
- feat = 89
- Wall time: 37min 47s
- cv = 0.66015
- LB = 0.6247

#### baseline_5
- feat = 74
- Wall time: 39min 18s
- cv = 0.66859
- LB = 0.6230

In [51]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [71]:
models = models_list_list[2]

In [72]:
feature_importance(models[0]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,340,495,553,456,1844
263,mean_price,289,420,504,327,1540
264,mean_price_per_cat,294,404,508,326,1532
259,register_number,221,412,455,333,1421
262,total,250,354,458,336,1398
261,date_rank,238,341,401,302,1282
260,spend_time,226,299,418,250,1193
258,week_time,219,314,351,233,1117
256,timezone_count,194,284,363,255,1096
257,date_diff,199,281,323,277,1080


In [73]:
feature_importance(models[1]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,74,70,190,273,607
264,mean_price_per_cat,69,59,149,241,518
259,total,46,65,159,217,487
263,mean_price,63,53,149,190,455
261,date_diff,53,67,142,184,446
258,register_number,43,46,147,181,417
262,date_rank,56,50,102,175,383
257,spend_time,41,43,128,167,379
260,timezone_count,47,54,104,158,363
255,week_time,35,36,118,145,334


In [74]:
feature_importance(models[2]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,260,262,290,299,1111
264,mean_price_per_cat,230,241,242,252,965
263,mean_price,214,215,260,237,926
259,register_number,175,192,263,213,843
262,total,208,194,208,221,831
261,date_diff,189,190,233,207,819
260,date_rank,179,186,218,224,807
257,spend_time,163,178,190,226,757
258,week_time,173,189,176,215,753
256,timezone_count,149,169,196,192,706


In [75]:
feature_importance(models[3]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,381,353,516,421,1671
263,date_rank,354,281,404,365,1404
264,mean_price_per_cat,359,270,402,348,1379
262,mean_price,343,278,389,360,1370
260,total,305,246,444,339,1334
261,register_number,316,231,417,333,1297
257,spend_time,279,213,342,319,1153
256,date_diff,270,230,338,278,1116
258,timezone_count,284,183,313,271,1051
259,week_time,289,198,272,276,1035


In [76]:
feature_importance(models[4]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,279,294,299,251,1123
264,date_rank,220,238,228,193,879
263,date_diff,184,216,217,165,782
260,mean_price,155,204,210,163,732
262,mean_price_per_cat,168,189,179,164,700
261,total,162,165,191,147,665
257,register_number,140,182,162,139,623
259,spend_time,140,157,165,126,588
258,week_time,140,164,127,127,558
253,timezone_count,119,147,154,133,553


In [77]:
feature_importance(models[5]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,610,730,382,716,2438
264,total,572,610,295,581,2058
261,mean_price,520,626,297,560,2003
263,register_number,540,626,265,564,1995
262,mean_price_per_cat,539,575,284,579,1977
260,date_rank,466,504,310,487,1767
259,date_diff,448,567,258,454,1727
258,spend_time,441,473,251,464,1629
256,week_time,397,483,246,409,1535
255,timezone_count,392,474,250,388,1504


In [78]:
feature_importance(models[6]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,450,373,531,394,1748
262,register_number,395,315,511,322,1543
261,total,395,306,480,337,1518
263,mean_price,404,308,459,314,1485
264,mean_price_per_cat,405,267,454,303,1429
260,date_diff,377,277,382,262,1298
258,date_rank,342,259,380,252,1233
259,spend_time,342,248,394,241,1225
257,timezone_count,329,243,368,244,1184
256,week_time,304,233,353,252,1142


In [79]:
feature_importance(models[7]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,435,698,482,492,2107
263,mean_price,413,708,471,484,2076
262,register_number,397,739,431,427,1994
264,total,421,698,430,393,1942
260,mean_price_per_cat,358,685,452,398,1893
259,date_rank,351,572,353,419,1695
257,date_diff,317,628,373,369,1687
261,spend_time,363,552,336,367,1618
256,timezone_count,309,532,382,361,1584
258,week_time,322,537,321,334,1514


In [80]:
feature_importance(models[8]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,276,192,188,292,948
262,date_rank,204,156,171,241,772
264,mean_price_per_cat,217,132,158,237,744
263,date_diff,207,134,150,206,697
261,mean_price,201,119,120,243,683
260,total,187,119,128,229,663
259,register_number,185,102,117,242,646
258,spend_time,169,82,102,186,539
257,week_time,163,72,102,195,532
256,timezone_count,151,81,89,192,513


In [81]:
feature_importance(models[9]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,250,237,417,353,1257
264,date_rank,227,200,340,288,1055
262,mean_price,192,173,396,290,1051
263,mean_price_per_cat,201,163,398,283,1045
260,total,189,181,355,281,1006
259,register_number,181,157,335,279,952
261,date_diff,191,135,310,277,913
257,spend_time,149,138,320,245,852
258,week_time,170,142,294,232,838
255,timezone_count,144,153,279,229,805


In [82]:
feature_importance(models[10]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,141,407,222,197,967
260,mean_price_per_cat,80,357,144,158,739
264,mean_price,104,346,174,107,731
262,total,93,311,173,124,701
263,date_diff,93,309,138,135,675
255,register_number,71,345,142,102,660
261,date_rank,87,283,158,123,651
254,timezone_count,68,285,162,112,627
257,week_time,71,290,139,108,608
259,spend_time,77,274,98,111,560


In [83]:
feature_importance(models[11]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,290,384,297,621,1592
264,mean_price_per_cat,229,327,225,534,1315
260,total,198,349,190,576,1313
261,register_number,201,314,194,599,1308
263,mean_price,220,338,196,539,1293
262,date_rank,213,281,228,489,1211
259,date_diff,187,271,209,445,1112
258,week_time,169,241,189,461,1060
256,spend_time,161,232,185,474,1052
255,timezone_count,153,229,139,403,924


In [84]:
feature_importance(models[12]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,556,599,520,380,2055
263,mean_price,476,495,471,343,1785
264,mean_price_per_cat,487,527,428,333,1775
261,register_number,448,494,451,332,1725
262,total,448,480,381,302,1611
258,date_rank,401,432,389,319,1541
259,date_diff,412,446,351,283,1492
260,spend_time,425,424,359,232,1440
257,week_time,366,402,325,315,1408
256,timezone_count,340,407,336,247,1330


In [85]:
feature_importance(models[13]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,162,158,231,507,1058
262,register_number,125,125,217,488,955
264,mean_price_per_cat,142,139,200,449,930
263,total,139,145,202,432,918
261,mean_price,122,109,182,414,827
259,date_rank,113,115,165,346,739
255,timezone_count,81,97,167,380,725
257,week_time,94,98,170,359,721
260,date_diff,120,101,156,294,671
258,spend_time,111,99,117,324,651


In [86]:
feature_importance(models[14]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
265,doy,149,286,190,255,880
262,mean_price_per_cat,100,223,129,218,670
256,mean_price,79,227,131,230,667
263,date_rank,112,195,136,206,649
264,date_diff,115,198,145,178,636
261,total,93,204,122,175,594
259,register_number,82,204,120,188,594
254,spend_time,74,201,109,148,532
253,week_time,74,170,94,158,496
257,timezone_count,80,157,99,145,481
